# VIMS Datathon 2023

In [1]:
from ultralytics import YOLO
import os
import json
import numpy as np
from collections import defaultdict

path_imgs = "./Test_Dataset_100/test_images" #Path to test images

path_gt_labels = "./Test_Dataset_100/test_2.json" # Path to gt_json file

model = YOLO('./model/bestYOLOv8Lresult.pt')  # path to the model

## Predictions

In [2]:
test_files = os.listdir(path_imgs)

coco_dt = []
for file in test_files:
    result = model(os.path.join(path_imgs,file))
    boxes = result[0].boxes  # Boxes object for bbox outputs 
    #print(results)
    #print(file)
    #print(boxes.xyxy)
    #print(boxes.cls)
    #print(boxes.conf)
    for k in range(len(boxes.cls)):
        A, B, C, D  = boxes.xyxy[k].tolist()
        bbox = [A, B, (C-A), (D-B)]
        coco_dt.append({
              'image_id': file,
              'category_id': boxes.cls[k].cpu().detach().numpy().item(),
              #'bbox': boxes.xyxy[k].tolist(),
              'bbox': bbox,
              'score': boxes.conf[k].cpu().detach().numpy().item()
              })
    
#print(coco_dt)

with open("./detections.json", "w") as outfile:
    json.dump(coco_dt, outfile)


image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/04804.jpg: 640x480 1 Crane, 47.5ms
Speed: 2.1ms preprocess, 47.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02147.jpg: 480x640 1 Scraper, 44.9ms
Speed: 1.1ms preprocess, 44.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02132.jpg: 384x640 1 Excavator, 1 Truck, 47.4ms
Speed: 0.8ms preprocess, 47.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/09335.jpg: 384x640 1 Excavator, 1 Truck, 7.1ms
Speed: 0.8ms preprocess, 7.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/03168.jpg: 640x384 2 Excavators, 46.1ms
Speed: 1.1ms preprocess, 46.1ms infe


image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/06505.jpg: 384x640 1 Scraper, 6.6ms
Speed: 0.7ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/07944.jpg: 384x640 3 Excavators, 3 Trucks, 6.6ms
Speed: 0.8ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02263.jpg: 384x640 2 Excavators, 1 Truck, 6.6ms
Speed: 0.7ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/01862.jpg: 480x640 1 Excavator, 7.8ms
Speed: 1.0ms preprocess, 7.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/03627.jpg: 384x640 1 Excavator, 1 Truck, 6.9ms
Speed: 0.8ms preprocess, 6.9

Speed: 0.7ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/ppe_0031.jpg: 384x640 1 worker, 6.6ms
Speed: 0.8ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/00814.jpg: 384x640 1 Excavator, 6.6ms
Speed: 0.8ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02722.jpg: 384x640 1 Crane, 4 Excavators, 1 Truck, 6.6ms
Speed: 0.8ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/05542.jpg: 384x640 1 Compactor, 6.6ms
Speed: 0.8ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dat

## Mapping with Ground Truth

In [3]:
predict_json = "./detections.json"

with open(predict_json) as json_file:
    predictions = json.load(json_file)

gt_json = path_gt_labels
with open(gt_json) as json_file:
    gts = json.load(json_file)

gt_ims = {}

for im in gts['images']:
    gt_ims[im['file_name']] = im['id']


for prediction in predictions:
    if prediction['image_id'] in gt_ims:
        prediction['image_id']= gt_ims[prediction['image_id']]

gt_cats = {}
#dt_cats = {'Crane': 1, 'Excavator': 2, 'Bulldozer': 3, 'Scraper': 4, 'Truck': 5, 'Compactor': 6, 'worker': 7}

for cat in gts['categories']:
    gt_cats[cat['name']] = cat['id']

print(gt_cats)
for prediction in predictions:
    if prediction['category_id'] == 1:
        prediction['category_id']= float(gt_cats['Crane'])
    elif prediction['category_id'] == 2:
        prediction['category_id']= float(gt_cats['Excavator'])
    elif prediction['category_id'] == 3:
        prediction['category_id']= float(gt_cats['Bulldozer'])
    elif prediction['category_id'] == 4:
        prediction['category_id']= float(gt_cats['Scraper'])
    elif prediction['category_id'] == 5:
        prediction['category_id']= float(gt_cats['Truck'])
    elif prediction['category_id'] == 7:
        prediction['category_id']= float(gt_cats['worker'])
    else:
        if "Compactor" in gt_cats.keys():
            prediction['category_id']= float(gt_cats['Compactor'])
        else:
            prediction['category_id'] = 0
    

with open("detections_new.json", "w") as outfile:
    json.dump(predictions, outfile)

{'Crane': 1, 'Excavator': 2, 'Bulldozer': 3, 'Scraper': 4, 'Truck': 5, 'worker': 7}


## Evaluation: mAP

In [4]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

predict_json_new = "./detections_new.json"

coco_gt = COCO(path_gt_labels)
coco_dt = coco_gt.loadRes(predict_json_new)
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Get mAP
mAP = coco_eval.stats[1]
print("\n mAP (Overall)", mAP)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.917
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.989
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.971
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.783
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.941
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.748
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.921
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

## Evaluation: mean IoU

In [5]:
predict_json_new = "./detections_new.json"

with open(predict_json_new) as json_file:
    predictions = json.load(json_file)

gt_categories_id = list(gt_cats.values())
gt_categories_name = list(gt_cats.keys())

cls_iou = defaultdict(list)
mIoU = {}
overall_miou = []

for pred in predictions:
    iou = coco_eval.computeIoU(pred['image_id'],pred['category_id'])
    if len(iou)>0:
        iou = np.max(iou)
        for cats in gt_categories_id:
            if pred['category_id'] == cats:
                cls_iou[cats].append(iou)

for cats in gt_categories_id:
    cls_miou = np.nanmean(cls_iou[cats])
    mIoU[cats]=cls_miou
    print("IoU of Class{}_{}: {}".format(cats, gt_categories_name[gt_categories_id.index(cats)], cls_miou))
    overall_miou.append(cls_miou)
overall_miou = np.nanmean(overall_miou)

print("\nmeanIoU (Overall):", overall_miou)
mIoU["Overall"]=overall_miou
print("\nMean IoU", mIoU)

IoU of Class1_Crane: 0.967513972057842
IoU of Class2_Excavator: 0.9601294157022492
IoU of Class3_Bulldozer: 0.9576445557302979
IoU of Class4_Scraper: 0.971295425935968
IoU of Class5_Truck: 0.9502396226213781
IoU of Class7_worker: 0.9669954908158426

meanIoU (Overall): 0.962303080477263

Mean IoU {1: 0.967513972057842, 2: 0.9601294157022492, 3: 0.9576445557302979, 4: 0.971295425935968, 5: 0.9502396226213781, 7: 0.9669954908158426, 'Overall': 0.962303080477263}


## Visualization

In [6]:
test_files = os.listdir(path_imgs)

for file in test_files:
    source = os.path.join(path_imgs,file)
    model.predict(source, save=True)


image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/04804.jpg: 640x480 1 Crane, 7.7ms
Speed: 0.8ms preprocess, 7.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02147.jpg: 480x640 1 Scraper, 7.8ms
Speed: 0.7ms preprocess, 7.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02132.jpg: 384x640 1 Excavator, 1 Truck, 7.0ms
Speed: 0.6ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/09335.jpg: 384x640 1 Excavator, 1 Truck, 6.6ms
Speed: 0.6ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1

Speed: 0.6ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02527.jpg: 384x640 1 Bulldozer, 6.6ms
Speed: 0.5ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/09731.jpg: 384x640 1 Excavator, 2 Trucks, 6.6ms
Speed: 0.7ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/07627.jpg: 384x640 1 Excavator, 1 Truck, 6.6ms
Speed: 0.6ms preprocess, 6.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/00118.jpg: 384x640 1 Excavator, 2 Truck

Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/04353.jpg: 512x640 1 Crane, 8.2ms
Speed: 1.2ms preprocess, 8.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/04780.jpg: 384x640 1 Excavator, 7.4ms
Speed: 0.6ms preprocess, 7.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/02567.jpg: 384x640 1 Excavator, 7.1ms
Speed: 0.8ms preprocess, 7.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/00101.jpg: 480x640 1 Crane, 8.2ms
Speed: 0.7ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect

Speed: 0.6ms preprocess, 7.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/04499.jpg: 384x640 1 Excavator, 7.1ms
Speed: 0.7ms preprocess, 7.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /home/aritra/datathon_2023/yolov8/Test_Dataset_100/test_images/05161.jpg: 448x640 1 Crane, 8.1ms
Speed: 1.1ms preprocess, 8.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
